# Prediction with ForceRegression model

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_kedro
import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot
import kedro

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

In [ ]:
id = "22774"
df = catalog.load(f"{id}.data")
df_smooth = catalog.load(f"{id}.data_ek_smooth")
model = catalog.load("force_regression.model")

In [ ]:
result = model.simulate(df_=df_smooth)

In [ ]:
dataframes = {
'model test' : df,
'simulation (model test)' : result.result,    
}

fig,ax=plt.subplots()
track_plots(dataframes=dataframes, lpp=model.ship_parameters['L'], beam=model.ship_parameters['B'], ax=ax);

plot(dataframes, keys=result.result.columns);